Reaction Diffusion
================

This example demonstrates constructing a simulation with reaction diffusion. 
Each particle carries a concentration of two species "U" and "V". 
Species concentrations change according to inter-particle transport (diffusion) 
and intra-particle interactions (reaction). 

Inter-particle diffusion occurs using typical Tissue Forge flux transport. 
Intra-particle reactions are specified in Antimony, which Tissue Forge 
converts to SBML using integrated libAntimony and executes using integrated libRoadRunner. 
In general, an arbitrary number of reaction kinetics models can be defined and attached 
to particles on the basis of particle and particle type. 

Tissue Forge supports most libRoadRunner features. This example demonstrates how to 
define a reaction kinetics model, attach it to a particle type, and selectively 
use different libRoadRunner integrators on the basis of individual particle and model. 

Reaction System
---------------------

Define the Schnakenberg system in Antimony with variables "U" and "V". The system will be the reaction term of the reaction diffusion system. 

In [1]:
schnakenberg_string = '''
J1: -> U; g * (U * U * V - U + a);
J2: -> V; g * (- U * U * V + b);

a = 0.1;
b = 0.9;
g = 1.0;
U = 0;
V = 0;
'''

Basic Setup
---------------

Declare a simulation domain with a single particle type. Declare species on the particle type of the same name as the variables in the reaction system. Attach the reaction system to the particle type and map species names to reaction system variable names. Use the universe time step as the reaction system time step. 

In [2]:
import tissue_forge as tf

tf.init(dt=0.01,
        dim=[20, 10, 10],
        cells=[6, 3, 3],
        bc={'x': 'free_slip', 'y': 'free_slip', 'z': 'free_slip'})


class AType(tf.ParticleTypeSpec):
    radius = 0.1
    species = ['U', 'V']
    style = {"colormap": {"species": "U", "range": (0.0, 3.0)}}
    reactions = {'schnakenberg': {'antimony_string': schnakenberg_string,
                                  'name_map': [('U', 'U'), ('V', 'V')],
                                  'step_size': tf.Universe.dt
                                  }}

A = AType.get()

Diffusion
------------

Define flux transport between particles, where the species "V" diffuses much faster than the species "U". 

In [3]:
diff_rel = 20.0
diff_cf = 10.0 / diff_rel
flux_cutoff = 0.28
tf.Fluxes.flux(A, A, "U", diff_cf, cutoff=flux_cutoff)
tf.Fluxes.flux(A, A, "V", diff_cf * diff_rel, cutoff=flux_cutoff)

<tissue_forge.tissue_forge.Fluxes; proxy of <Swig Object of type 'TissueForge::Fluxes *' at 0x103f9a060> >

Deterministic Reaction Diffusion on the Sphere
--------------------------------------------------------

Make an icosphere and place it at an offset from the center of the universe. Randomly initialize the species "U", and initial the species "V" as zero. 

In [4]:
import numpy as np

sphere_radius = 4.0
offset = tf.FVector3(sphere_radius + A.radius, 0, 0)
for pos in tf.icosphere(5, 0, 2 * np.pi)[0]:
    p = A(position=tf.Universe.center - offset + pos * sphere_radius)
    p.species.U.value = np.random.random()
    p.species.V.value = 0.0

Stochastic Reaction Diffusion on the Sphere
--------------------------------------------------------

Make another icosphere and place it at the same offset but with opposite sense from the center of the universe. Randomly initialize the species "U", and initial the species "V" as zero. Use the Gillespie Stochastic Solver Algorithm for all particles on the sphere. 

In [5]:
for pos in tf.icosphere(5, 0, 2 * np.pi)[0]:
    p = A(position=tf.Universe.center + offset + pos * sphere_radius)
    p.species.U.value = np.random.random()
    p.species.V.value = 0.0
    p.species.reactions['schnakenberg'].integrator_name = "gillespie"

In [6]:
tf.system.camera_view_front()
tf.system.camera_zoom_to(-25)
tf.system.decorate_scene(False)
tf.show()